# Proyecto: Predicción de Anemia en Mujeres

## 📊 Perfil del Proyecto

## Fase 1. Business Understanding

La anemia en mujeres en edad fértil representa un desafío significativo de salud pública en Perú, con una prevalencia nacional que oscila entre 30-40% según reportes del Ministerio de Salud. La detección temprana es crucial para implementar intervenciones efectivas, pero los métodos actuales de screening presentan limitaciones en cobertura y eficiencia. Este proyecto busca desarrollar un modelo predictivo que permita identificar mujeres en riesgo de anemia utilizando datos disponibles de encuestas nacionales de salud.

### 1. Objetivos
Objetivo Principal:
Desarrollar un modelo de clasificación binaria que prediga la presencia de anemia en mujeres peruanas en edad fértil (15-49 años) con un F1-Score ≥ 0.85.

#### Objetivos Específicos:

1. Identificar los cinco principales factores de riesgo asociados con anemia
2. Establecer un sistema de priorización para intervenciones de salud pública

3. Reducir los costos operativos de screening en al menos 30%

4. Proporcionar recomendaciones accionables para programas de prevención

### Criterios de Éxito
#### Métricos Técnicos:

- F1-Score: ≥ 0.85 en validación cruzada

- Precisión: ≥ 0.80

- Sensibilidad: ≥ 0.85

- AUC-ROC: ≥ 0.90

#### Métricas de Negocio:

- Identificación de al menos 5 factores de riesgo con significancia estadística

- Reducción del 30% en costos de screening mediante focalización

- Tasa de detección temprana del 80% en población de riesgo

- Validación y aceptación por parte de expertos en salud pública

## Fase 2. Data Understanding

### 📥 2. Fuentes de Datos

#### 📁 Descripción del Dataset

El proyecto utiliza dos datasets principales de encuestas de salud peruanas:

1. `RECH0.csv`: Datos demográficos y socioeconómicos de hogares

    - Registros: 37,390

    - Variables: 44 columnas

    - Período de recolección: 2024

2. `RECH5.csv`: Datos de salud y nutrición de mujeres en edad fértil

    - Registros: 38,488

    - Variables: 40 columnas

    - Período de recolección: 2024

#### Variables Clave Identificadas
##### Variables Demográficas (RECH0):

- `IDHOGAR`: Identificador único del hogar

- `EDAD_MUJER`: Edad de la mujer en años

- `NIVEL_EDUCATIVO`: Máximo nivel educativo alcanzado

- `REGION`: Región o departamento de residencia

- `AREA_URBANA_RURAL`: Clasificación urbano/rural

- `IDIOMA_MATERNO`: Idioma principal hablado

- `ACCESO_SERVICIOS_BASICOS`: Indicador de acceso a servicios

##### Variables de Salud (RECH5):

- `HEMOGLOBINA`: Nivel de hemoglobina en g/dL

- `IMC`: Índice de Masa Corporal

- `PESO_KG`: Peso en kilogramos

- `TALLA_CM`: Talla en centímetros

- `EMBARAZO_ACTUAL`: Estado de embarazo actual

- `SUPLEMENTOS_HIERRO`: Consumo de suplementos de hierro

- `ANEMIA_CLASIFICACION`: Clasificación actual de anemia

#### Estrategia de Integración
Los datasets se unirán mediante la variable IDHOGAR, permitiendo consolidar información demográfica con datos de salud para el análisis integral.

#### Consideraciones Éticas
- Todos los datos personales están anonimizados

- Cumplimiento de la Ley de Protección de Datos Personales peruana

- Uso exclusivo para investigación en salud pública

- Aprobación del comité de ética correspondiente



In [ ]:
import pandas as pd
import numpy as np

'''
df_p1 = pd.read_csv("in_data_cruda_p1_27072025.csv",
                    encoding='utf-8',      # o 'latin-1' si hay caracteres especiales
                    sep=';',               # separador (por defecto es coma)
                    header=0,              # fila que contiene los nombres de columnas
                    index_col=None)        # si quieres usar una columna como índice
'''
df_1 = pd.read_csv("data/968-Modulo1629/RECH0_2024.csv")

/opt/notebooks/md-project


In [13]:
df_2 = pd.read_csv("data/968-Modulo1638/RECH5_2024.csv")

In [15]:
# Resumen general Dataset RECH0
df_1.info()
print('df.columns:',df_1.columns)
print('df.shape:',df_1.shape)
df_1.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37390 entries, 0 to 37389
Data columns (total 44 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID1         37390 non-null  int64  
 1   HHID        37390 non-null  int64  
 2   HV000       37390 non-null  object 
 3   HV001       37390 non-null  int64  
 4   HV002       37390 non-null  int64  
 5   HV002A      37390 non-null  int64  
 6   HV003       37390 non-null  int64  
 7   HV004       37390 non-null  int64  
 8   HV007       37390 non-null  int64  
 9   HV008       37390 non-null  int64  
 10  HV009       37390 non-null  int64  
 11  HV010       37390 non-null  int64  
 12  HV011       37390 non-null  int64  
 13  HV012       37390 non-null  int64  
 14  HV013       37390 non-null  int64  
 15  HV014       37390 non-null  int64  
 16  HV015       37390 non-null  int64  
 17  HV017       37390 non-null  int64  
 18  HV020       37390 non-null  int64  
 19  HV021       37390 non-nul

,ID1,HHID,HV000,HV001,HV002,HV002A,HV003,HV004,HV007,HV008,...,HV043,HV044,UBIGEO,HV022,HV005,CODCCPP,NOMCCPP,LATITUDY,LONGITUDX,NCONGLOME1
0,2024,325502001,PE6,3255,20,1,0,3255,2024,1493,...,0,1,10101,2,0,1,CHACHAPOYAS,-6.225832,-77.861302,7076
1,2024,325503101,PE6,3255,31,1,2,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
2,2024,325503901,PE6,3255,39,1,1,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
3,2024,325504001,PE6,3255,40,1,1,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
4,2024,325504701,PE6,3255,47,1,2,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
5,2024,325505001,PE6,3255,50,1,1,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
6,2024,325505301,PE6,3255,53,1,1,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
7,2024,325508901,PE6,3255,89,1,1,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
8,2024,325509301,PE6,3255,93,1,2,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
9,2024,325509701,PE6,3255,97,1,1,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076


In [16]:
# Resumen general Dataset RECH5
df_2.info()
print('df.columns:',df_2.columns)
print('df.shape:',df_2.shape)
df_2.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38488 entries, 0 to 38487
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID1     38488 non-null  int64 
 1   HHID    38488 non-null  int64 
 2   HA0     38488 non-null  int64 
 3   HA1     38488 non-null  int64 
 4   HA2     38488 non-null  int64 
 5   HA3     38488 non-null  int64 
 6   HA4     38488 non-null  int64 
 7   HA5     38488 non-null  int64 
 8   HA6     38488 non-null  int64 
 9   HA11    38488 non-null  int64 
 10  HA12    38488 non-null  int64 
 11  HA12A   38488 non-null  int64 
 12  HA12B   38488 non-null  int64 
 13  HA13    38488 non-null  int64 
 14  HA32    38488 non-null  int64 
 15  HA33    38488 non-null  int64 
 16  HA35    38488 non-null  int64 
 17  HA40    38488 non-null  int64 
 18  HA41    38488 non-null  int64 
 19  HA50    38488 non-null  int64 
 20  HA51    38488 non-null  object
 21  HA52    38488 non-null  int64 
 22  HA53    38488 non-null

,ID1,HHID,HA0,HA1,HA2,HA3,HA4,HA5,HA6,HA11,...,HA60,HA61,HA62,HA63,HA64,HA65,HA66,HA67,HA68,HA69
0,2024,325503101,2,38,704,1525,301,-188,9315,108,...,,,,,,1,2,5,2,
1,2024,325503101,4,15,541,1518,591,-156,9355,16,...,,,,,,1,2,3,2,
2,2024,325503901,2,17,498,1602,2931,-54,9798,-101,...,,,,,,1,2,5,2,
3,2024,325504701,2,32,653,1443,6,-325,8814,146,...,,,,,,1,2,5,2,
4,2024,325505001,1,36,689,1486,57,-253,9077,122,...,,,,,,1,1,6,1,
5,2024,325505301,1,32,729,1570,1303,-112,9590,127,...,,,,,,1,1,6,1,
6,2024,325508901,2,34,573,1533,405,-175,9364,-11,...,,,,,,1,2,5,2,
7,2024,325509301,1,24,669,1547,655,-151,9450,139,...,,,,,,1,3,5,3,
8,2024,325509301,2,18,676,1515,203,-205,9254,171,...,,,,,,1,3,1,3,
9,2024,325509701,2,30,504,1533,405,-175,9364,-99,...,,,,,,1,2,3,2,
